In [3]:
import numpy as np
import tensorflow as tf

In [4]:
y_true = np.array([[0.,1.,1.],[1.,1.,0.]])
y_pred = np.array([[2.,1.,1.],[1.,1.,1.]])

# tf输入Numpy数据时会自动转换为Tensor来处理
# 显式处理
# y_true= tf.convert_to_tensor(y_true)
# y_pred= tf.convert_to_tensor(y_pred)
y_true

array([[0., 1., 1.],
       [1., 1., 0.]])

### logistic regression loss

In [3]:
lll_loss = tf.nn.log_poisson_loss(targets=y_true,log_input=tf.log(y_pred))

### sigmoid cross entropy loss

In [4]:
"""
https://www.jianshu.com/p/cf235861311b
z * -log(sigmoid(x)) + (1 - z) * -log(1 - sigmoid(x))
sigmoid_cross_entropy_with_logits:
对输入的logits先通过sigmoid函数计算，再计算它们的交叉熵，但是它对交叉熵的计算方式进行了优化，使得的结果不至于溢出。
"""
sigmoid_ce_logits = tf.nn.sigmoid_cross_entropy_with_logits(labels=y_true,
                                        logits=y_pred)
sigmoid_ce_logits_loss = tf.reduce_mean(sigmoid_ce_logits)

y_hat_sigmoid = tf.nn.sigmoid(y_pred)
sigmoid_total_loss = tf.reduce_mean(-y_true*tf.log(y_hat_sigmoid)-(1-y_true)*tf.log(1-y_hat_sigmoid))
# sigmoid_total_loss = tf.reduce_mean(-tf.reduce_sum(y_true * tf.log(y_hat_sigmoid), [1]))

### softmax_cross_entropy

In [5]:
"""
https://www.jianshu.com/p/6c9b0cc6978b
If you interpret the scores in y_hat as unnormalized log probabilities, then they are logits.
"""
"""softmax loss"""
softmax_ce_logits = tf.nn.softmax_cross_entropy_with_logits(labels=y_true,
                                        logits=y_pred)
softmax_ce_logits_loss = tf.reduce_mean(softmax_ce_logits)

y_hat_softmax = tf.nn.softmax(y_pred)
softmax_total_loss = tf.reduce_mean(-tf.reduce_sum(y_true * tf.log(y_hat_softmax), [1]))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



In [6]:
with tf.Session() as sess:
    # print(sess.run(y))  # ERROR: 此处x还没有赋值.

    # rand_array = np.random.rand(1024, 1024)
    print(sess.run(lll_loss))

    """sigmoid"""
    print(sess.run(sigmoid_ce_logits_loss))
    print(sess.run(sigmoid_total_loss))
    """softmax"""
    print(sess.run(softmax_ce_logits_loss))
    print(sess.run(softmax_total_loss))

[[2. 1. 1.]
 [1. 1. 1.]]
0.7822060747723478
0.7822060747723475
2.650057002600161
2.6500570026001613


### Contrastive Loss

In [3]:
from keras import backend as K
import tensorflow as tf
import numpy as np
from keras.losses import *

In [5]:
a = np.array([2.0, 5]) 
b = np.array([3.4, 5]) 
res = np.exp(a)
res

array([  7.3890561, 148.4131591])

In [8]:
### MSE

In [6]:
from sklearn.metrics import r2_score
"""
MSE
1 - mean_squared_error(y_true,y_predict)/np.var(y_true)
"""
r2_score(a,b)

0.5644444444444445

In [7]:
from sklearn.metrics import mean_squared_error
mean_squared_error(a,b)

0.9799999999999999

In [9]:
def exp_loss(y_true,y_pred):
    return K.mean(K.exp(-y_true*y_pred))

### Triple Loss

In [ ]:
def cos_distance(vects):
    x, y = vects
    fz = K.batch_dot(x,y,axes=1)
    fm1 = K.sum(K.square(x), axis=1, keepdims=True)
    fm2 = K.sum(K.square(y), axis=1, keepdims=True)
    fm = K.sqrt(fm1*fm2)
    return fz/K.maximum(fm, K.epsilon())

def triplet_loss(y_true, y_pred):
    """
    一次看三个样本
    Anchor, Positive, Negative
    L=max(d(a,p)−d(a,n)+margin,0)
    """
    MARGIN = 1.0  # 超参
    l = 768
# #     anc, pos, neg = y_pred[:, 0:128], y_pred[:, 128:256], y_pred[:, 256:]
    anc, pos, neg = y_pred[:, 0:l], y_pred[:, l:l*2], y_pred[:, l*2:]

#     # 欧式距离
    pos_dist = 1-K.abs(cos_distance((anc,pos)))
    neg_dist = 1-K.abs(cos_distance((anc,neg)))
#     pos_dist = K.sum(K.square(anc - pos), axis=-1, keepdims=True)
#     neg_dist = K.sum(K.square(anc - neg), axis=-1, keepdims=True)
#     pos_dist = y_pred[:, 0:1]
#     neg_dist = y_pred[:,1:]
    basic_loss = pos_dist - neg_dist + MARGIN
#     loss = basic_loss
    loss = K.maximum(basic_loss, 0.0)
#     print "[INFO] model - triplet_loss shape: %s" % str(loss.shape)
    return loss

### contrastive_loss

In [10]:
def contrastive_loss(y_true, y_pred):
    """
    一次看两个样本
    对比损失
    y_pred: 两个样本欧式距离（0,1），距离越接近0，两个样本越相似
    y_true：y_true = 1 匹配，0： 不匹配
    Contrastive loss from http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
    """
    margin = 1
    return K.mean(y_true * K.square(y_pred) + (1 - y_true) * K.square(K.maximum(margin - y_pred, 0)))

In [11]:
## emulation data
y_true = tf.constant([0.0, 0.0, 0.0, 0.0, 0.0])
y_pred = tf.constant([1.0, 1.0, 1.0, 1.0, 1.0])

In [12]:
K.eval(contrastive_loss(y_true,y_pred))

0.0

In [13]:
# binary_crossentropy
K.eval(binary_crossentropy(y_true, y_pred))

15.942385

In [14]:
K.eval(exp_loss(y_true,y_pred))

1.0

In [3]:
import matplotlib.pyplot as plt
def show_loss(history):
    """
    loss 可视化
    """
#     print(history.history.keys())
    fig = plt.figure()
    plt.plot(history.history['acc'])
    plt.plot(history.history['loss'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['acc',"loss"], loc='upper left')
# show_loss()